In [1]:
# One Pickle for NSE

# STATUS: Completed
# Run-time: 2 mins

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 3000, clientId=2)

In [29]:
%%time
import pandas as pd
import numpy as np
import itertools
import datetime
from math import sqrt, exp, log, erf
import os

#... assignments
exchange = 'NSE'
fspath = './zdata/'

putsigma = 1.5
callsigma = 2
maxdte = 65  # max expiry date for options

tradingdays = 252

blks = 50

#... Get risk-free rate from 91 day T-bills
rate_url = 'https://rbi.org.in/home.aspx'

li = pd.read_html(rate_url)
li_df = li[4].rename(columns = {0: 'Cat', 1: 'Values'})
li_val = li_df.loc[li_df.Cat == '91 day T-bills', 'Values']
rate = float((str(li_val).split('\n')[0].split('%')[0].split(' ')[-1:])[0])/100

#... Functions
#_____________

#... Error catching for list comprehension

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher
    Args: 
        (func) as the function
         (handle) as the lambda of function
         <*args | *kwargs> as arguments to the functions
    Outputs:
        output of the function | <np.nan> on error
    Usage:
        eggs = [1,3,0,3,2]
        [catch(lambda: 1/egg) for egg in eggs]'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan
        
#...function to get historical data
def get_hist(contract, duration):
    '''Gets 1-day bars of contracts for the duration specified
    Args:
        (contract) as obj
        (duration) as int
    Returns: dataframe of symbol, date, ohlc, avg and volume 
    '''
    
    # Prepare the duration
    strduration = str(duration) + ' D'
    
    # Extract the history
    hist = ib.reqHistoricalData(contract=contract, endDateTime='', 
                                    durationStr=strduration, barSizeSetting='1 day',  
                                                whatToShow='Trades', useRTH=True)
    
    df = util.df(hist)
    df.insert(0, column='symbol', value=contract.symbol)
    
    return df

#...function to get price and dividend ticker
def get_dividend_ticker(contract):
    '''Gets dividend ticker of the contract
    Arg: (contract) as a qualified contract object with conId
    Returns: ticker'''
    
    ib.reqMktData(contract, '456', snapshot=False, regulatorySnapshot=False) # request ticker stream

    ticker = ib.ticker(contract)
    
    # Ensure the ticker is filled
    while ticker.dividends is None:
        ib.sleep(1)

    ib.cancelMktData(contract)
       
    return ticker

#... Black-Scholes
# Ref: - https://ideone.com/fork/XnikMm - Brian Hyde

def get_bsm(undPrice, strike, dte, rate, volatility, divrate):
    ''' Gets Black Scholes output
    Args:
        (undPrice) : Current Stock Price in float
        (strike)   : Strike Price in float
        (dte)      : Days to expiration in float
        (rate)     : dte until expiry in days
        (volatility)    : Standard Deviation of stock's return in float
        (divrate)  : Dividend Rate in float
    Returns:
        (delta, call_price, put_price) as a tuple
    '''
    #statistics
    sigTsquared = sqrt(dte/365)*volatility
    edivT = exp((-divrate*dte)/365)
    ert = exp((-rate*dte)/365)
    d1 = (log(undPrice*edivT/strike)+(rate+.5*(volatility**2))*dte/365)/sigTsquared
    d2 = d1-sigTsquared
    Nd1 = (1+erf(d1/sqrt(2)))/2
    Nd2 = (1+erf(d2/sqrt(2)))/2
    iNd1 = (1+erf(-d1/sqrt(2)))/2
    iNd2 = (1+erf(-d2/sqrt(2)))/2

    #Outputs
    callPrice = round(undPrice*edivT*Nd1-strike*ert*Nd2, 2)
    putPrice = round(strike*ert*iNd2-undPrice*edivT*iNd1, 2)
    delta = Nd1

    return {'bsmCall': callPrice, 'bsmPut': putPrice, 'bsmDelta': delta}

#... build the symbols
#______________________

# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct', 'Mlot': 'lot'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

# separate indexes and equities, eliminate discards from df_paisa
indexes = ['NIFTY50', 'BANKNIFTY']
discards = ['NIFTYMID5', 'NIFTYIT', 'LUPIN']
equities = sorted([s for s in df_paisa.ibSymbol if s not in indexes+discards])

symbols = equities+indexes

c = [Stock(s, exchange) if s in equities else Index(s, exchange) for s in symbols]
contracts = ib.qualifyContracts(*c)

######   DATA LIMITER ######
contract = [c for c in contracts if c.symbol == 'AMARAJABA'][0]  # !!! DATA Limiter
#__________________________________________




#... get ohlc, with cumulative volatality and standard deviation
#_______________________________________________________________

df_ohlc = get_hist(contract, 365).set_index('date')

# get cumulative volatility
df_vol = pd.DataFrame(df_ohlc['close'].sort_index(ascending=[True, False]).pct_change().expanding(1).std(ddof=0)*sqrt(tradingdays))
df_vol.columns = ['volatility']

# get cumulative standard deviation
df_stdev = pd.DataFrame(df_ohlc['close'].sort_index(ascending=[True, False]).expanding(1).std(ddof=0))
df_stdev.columns = ['stdev']


df_ohlc1 = df_ohlc.join(df_vol)

df_ohlc2 = df_ohlc1.join(df_stdev)

#pickle the ohlc
df_ohlc2.to_pickle(fspath+contract.symbol+'_ohlc.pkl')

#... get the underlyings
#_______________________

ticker = get_dividend_ticker(contract)

df_und = util.df([ticker])

cols = ['contract', 'time', 'bid', 'bidSize', 'ask', 'askSize', 'last', 'lastSize', 
        'volume', 'open', 'high', 'low', 'close', 'dividends']
df_und = df_und[cols]

df_und = df_und.assign(undPrice=np.where(df_und['last'].isnull(), df_und.close, df_und['last']))

try: 
    divrate = df_und.dividends[0][0]/df_und.dividends[0][0]/df_und.undPrice
except (TypeError, AttributeError) as e:
    divrate = 0.0

df_und = df_und.assign(divrate=divrate)

df_und = df_und.assign(symbol=[c[1].symbol for c in df_und.contract.items()])

# get lot and margin
df_paisa1 = df_paisa[['ibSymbol', 'lot', 'marginpct']].rename({'ibSymbol': 'symbol'}, axis=1)

df_und1 = df_und.merge(df_paisa1)

df_und1.to_pickle(fspath+contract.symbol+'_und.pkl')

#... get the options
#___________________

# symbol
symbol = contract.symbol

# rights
right = ['P', 'C'] 

# chains
chain = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, futFopExchange='', 
                      underlyingConId=contract.conId, underlyingSecType=contract.secType)

srek = [list(itertools.product([symbol], right, l[0], l[1])) for l in [(es[0], es[1]) for es in [(c.expirations, c.strikes) for c in chain]]]

# options dataframe
df_opt = pd.DataFrame([x for s in srek for x in s], columns=['symbol', 'right', 'expiry', 'strike'])

df_opt = df_opt.assign(undPrice=df_opt.symbol.map(df_und.set_index('symbol')['undPrice']))

df_opt['dte'] = (df_opt.expiry.apply(util.parseIBDatetime) - datetime.datetime.now().date()).dt.days

df_opt.loc[df_opt.dte < 1, 'dte'] = 2 # Make the dte as 2 for 1 day-to-expiry to prevent bsm divide-by-zero error

# remove options that are more than maxdte
df_opt = df_opt[df_opt.dte <= maxdte].reset_index(drop=True)

# get the standard deviation based on days to expiry
df_opt = df_opt.assign(stdev=[df_ohlc2.iloc[i].stdev for i in df_opt.dte])

# weed out options within threshold of strike and dte
mask = (((df_opt.right == 'P') & (df_opt.strike < (df_opt.undPrice - df_opt.stdev * putsigma))) | \
       ((df_opt.right == 'C') & (df_opt.strike > (df_opt.undPrice + df_opt.stdev * callsigma))))

df_opt = df_opt.loc[mask, :].reset_index(drop=True)

# get the volatality based on days to expiry
df_opt = df_opt.assign(volatility=[df_ohlc2.iloc[i].volatility for i in df_opt.dte])

# get the divrate
divrate = df_und.divrate[0]

bsms = [get_bsm(undPrice, strike, dte, rate, volatility, divrate) 
        for undPrice, strike, dte, rate, volatility, divrate in 
        zip(df_opt.undPrice, df_opt.strike, df_opt.dte, itertools.repeat(rate), df_opt.volatility, itertools.repeat(divrate))]

df_bsm = pd.DataFrame(bsms)

df_opt1 = df_opt.join(df_bsm)
df_opt1['bsmPrice'] = np.where(df_opt1.right == 'P', df_opt1.bsmPut, df_opt1.bsmCall)
df_opt1['pop'] = np.where(df_opt1.right == 'C', 1-df_opt1.bsmDelta, df_opt1.bsmDelta)
df_opt1 = df_opt1.drop(['bsmCall', 'bsmPut', 'bsmDelta'], axis=1)

opts = [Option(s, e, k, r, x) for s, e, k, r, x in zip(df_opt1.symbol, df_opt1.expiry, df_opt1.strike, df_opt1.right, itertools.repeat(exchange))]

qual_opts = [catch(lambda: ib.qualifyContracts(cs)) for i in range(0, len(opts), blks) for cs in opts[i: i+blks]]

df_opt1['option'] = [x[0] if x != [] else np.nan for x in qual_opts] # make empty lists [] as nan

df_opt2 = df_opt1.dropna().reset_index(drop=True) # drop empty lists

# get the option prices
cs = list(df_opt2.option)

# [catch(lambda: ib.reqTickers(c).marketPrice()) for i in range(0, len(cs), 100) for c in cs[i: i+100]]
tickers = [ib.reqTickers(*cs[i: i+100]) for i in range(0, len(cs), 100)]

df_opt3 = df_opt2.assign(price=[t.marketPrice() for ts in tickers for t in ts])

lot = df_und1.lot[0]
marginpct = df_und1.marginpct[0]

df_opt3 = df_opt3.assign(rom=df_opt3.price/(marginpct*df_opt3.undPrice)*tradingdays/df_opt3.dte)
df_opt3 = df_opt3.sort_values(by='rom', ascending=False)

df_opt3.to_pickle(fspath+contract.symbol+'_opt.pkl')

Started to throttle requests
Stopped to throttle requests
Error 200, reqId 628: No security definition has been found for the request, contract: Option(symbol='AMARAJABA', lastTradeDateOrContractMonth='20190328', strike=380.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='AMARAJABA', lastTradeDateOrContractMonth='20190328', strike=380.0, right='P', exchange='NSE')
Error 200, reqId 677: No security definition has been found for the request, contract: Option(symbol='AMARAJABA', lastTradeDateOrContractMonth='20190328', strike=1120.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='AMARAJABA', lastTradeDateOrContractMonth='20190328', strike=1120.0, right='C', exchange='NSE')
Error 200, reqId 714: No security definition has been found for the request, contract: Option(symbol='AMARAJABA', lastTradeDateOrContractMonth='20190228', strike=1120.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='AMARAJABA', lastTradeDateOrContractMonth='20190228', strike=1120

Wall time: 1min 30s


In [28]:
band = ((df_opt3.strike < (df_opt3.undPrice - df_opt3.stdev*putsigma*2)) & (df_opt3.right == 'P') |
        (df_opt3.strike > (df_opt3.undPrice + df_opt3.stdev*callsigma*1.5)) & (df_opt3.right == 'C'))
df_opt3[(df_opt3.rom > 0.4) & band]

,symbol,right,expiry,strike,undPrice,dte,stdev,volatility,bsmPrice,pop,option,price,rom
69,AMARAJABA,C,20190131,800.0,746.7,2,4.976668,0.090140,0.00,1.000000,"Option(conId=339336236, symbol='AMARAJABA', la...",1.175,1.122085
41,AMARAJABA,P,20190228,700.0,746.7,28,14.647388,0.253009,4.27,0.847799,"Option(conId=343995395, symbol='AMARAJABA', la...",12.300,0.839006
88,AMARAJABA,C,20190228,800.0,746.7,28,14.647388,0.253009,5.29,0.810160,"Option(conId=343995428, symbol='AMARAJABA', la...",9.850,0.671887
26,AMARAJABA,P,20190131,720.0,746.7,2,4.976668,0.090140,0.00,1.000000,"Option(conId=339336208, symbol='AMARAJABA', la...",0.500,0.477483
91,AMARAJABA,C,20190228,820.0,746.7,28,14.647388,0.253009,2.68,0.890812,"Option(conId=343995438, symbol='AMARAJABA', la...",6.325,0.431440
